<a href="https://colab.research.google.com/github/casangi/ngcasa/blob/master/docs/ngcasa_calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calibration
--------- This section is very preliminary, 
with only the most basic calibration operations illustrated. 


## bandpass + gaincal : 
Use case : Solve for and applying calibration solutions in a sequence. 

Step (1) : Predict Model

Step (2) : Bandpass calibration, where data are averaged across time. Solutions are per channel.

Step (3) : Gain calibration, where data are averaged across frequency. Solutions are per time interval

Also, include an autoflag step on the gain solutions before applying, in Step (1).

This example is a simple situation of different data averaging required for each solve step. More complex sequences may be strung together with different data reshaping/processing and different ngcasa.calibration.solve_xxx() calls. 

In [0]:
# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

## Step (1) 

# Predict model visibilities (in lsrk frame) from a component list, and write to the DATA array in the XDS
cngi.vis.regridspw(vis_dataset)  # Convert from topo to lsrk
ngcasa.imaging.predict_modelvis_component(vis_dataset, component_list, arr_name='DATA') # De-grid and predict model
cngi.vis.regridspw(vis_dataset)  # Convert from lsrk back to data frame (topo)

## Step (2)

# Do time-averaging
vis_dataset_time_avg = cngi.vis.timeaverage(vis_dataset)

# Solve for gains ('solpars' encodes the solution type)
cal_dataset_bandpass = ngcasa.calibration.solve(vis_dataset_time_avg, solpars, use_arr_name='data')

# <Optionally run autoflag on the gain solutions to take out outliers>
# ngcasa.flagging.auto_clip(cal_dataset_bandpass)
# cngi.vis.applyflags(cal_dataset_bandpass)

# Apply the solutions to the original dataset
ngcasa.calibration.apply(vis_dataset, 
                         cal_dataset_bandpass, 
                         use_arr_name='data', 
                         out_arr_name='corrected_data')

## Step (3) 

# Do chan-averaging on original dataset
vis_dataset_chan_avg = cngi.vis.chanaverage(vis_dataset)

# Solve for gains ('solpars' encodes the solution type)
cal_dataset_gaincal = ngcasa.calibration.solve(vis_dataset_chan_avg, solpars, use_arr_name='corrected_data')

# Apply the solutions to the pre-calibrated dataset. Note the change in arr_name. 
ngcasa.calibration.apply(vis_dataset, 
                         cal_dataset_gaincal, 
                         use_arr_name='corrected_data', 
                         out_arr_name='corrected_data')

## OR..... Re-apply all the solutions to the original data...
#ngcasa.calibration.apply(vis_dataset, 
#                         [cal_dataset_bandpass, cal_dataset_gaincal], 
#                         use_arr_name='data',
#                         out_arr_name='corrected_data')

# Save calibrated data to disk
cngi.write_zarr(vis_dataset)


## Solutions requiring pre-apply of upstream solutions

Example : Pre-apply a pre-existing bandpass solution before averaging (to the MODEL data) and solve for time-varying gains. 

TBD : Do we need this mode of operation ? 

-- Since calibration solutions are obtained from a point source model (i.e. data / model ), the multiplication of g_i g_j\*  with Model visibilities is the same as dividing g_i g_j\* out of the Data visibilities.

-- Therefore, will the above solution (of applying solutions to the data and corrected_data column in a sequence) suffice ? Note that nothing is written to disk after each apply, but only written once at the very end.

TBD : If this doesn't cover all use cases, and we need an explicit pre-apply, need to define  ngcasa.calibration.pre_apply() to multiply 'MODEL' data with g_i g_j* from a list of input cal_datasets. 



## Polarization calibration (TBD)
TBD : Add example ?